# Demo: OpenAI Large Language Model - A RAG Sample

In this demo, we show how to use GPT3 model to analyze natural query, use knowledge base to search for more information and answer questions.

In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv("../config/environment_variables.env")

client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT_TEXT"), 
  api_key=os.environ.get("AZURE_OPENAI_KEY_TEXT"),  
  api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
)

CHAT_COMPLETIONS_MODEL = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")

### Start with a natural question

In [2]:
input = "Menurutmu siapakah orang yang akan memenangkan Pemilihan Presiden Indonesia 2024? Sertakan pula alasannya"

### Step 1: GPT3: What do I need to to answer this question?

In [3]:
system_prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Web Search: Use the web to find information

You need to follow STRICTLY below format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

Saya adalah AI yang didesain untuk memberikan informasi, namun saya tidak memiliki kemampuan untuk meramalkan masa depan atau memiliki opini pribadi. Oleh karena itu, saya tidak dapat memberikan prediksi tentang siapa yang akan memenangkan Pemilihan Presiden Indonesia 2024. Namun, saya dapat memberikan informasi umum tentang proses pemilihan presiden di Indonesia jika Anda tertarik. Apakah Anda ingin saya melakukannya?
Action: Research
Action Input: Pemilihan Presiden Indonesia 2024


### Step 2: Search web for more details 

In [5]:
# Use Azure BING Search API to get the results
import requests
import json
url = "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": os.environ.get('BING_SUBSCRIPTION_KEY')}
params = {"q": "2024 presidential election candidates and current political analysis in Indonesia"}
response = requests.get(url, headers=headers, params=params)

In [6]:
context = ''
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

### Collected information for further processing

In [7]:
print(context)

Prabowo Subianto will be the next president of Indonesia Last updated on February 14th 2024 Editor's note (February 14th 2024): This page has been updated with the results of the election.
Indonesia 2024 Presidential Candidates Analysis: Anies Baswedan — Muhaimin Iskandar | by Kajian Riset Strategis PPI London | Jan, 2024 | Medium. Kajian Riset Strategis PPI London. ·....
Indonesia: The 2024 Presidential Election Race Has Begun. Analysis. In Jakarta, Political Kingmaking Starts Now. Indonesia’s presidential election is less than a year away, and...
Three pairs of president and vice president candidates running for the Indonesia’s 2024 election. Galih Pradipta /Antara Foto November 24, 2023
Indonesia’s presidential candidates, from left, Ganjar Pranowo, Prabowo Subianto and Anies Baswedan after the first presidential candidates’ debate in Jakarta on December 12, 2023. Photo:...
Ary Hermawan, The University of Melbourne. Soeharto’s oligarchy, not Prabowo, is subverting Indonesia’s democr

In [8]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### Step 3: GPT3: What do I need to do to answer this question? (Again)

In [9]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

Menurut informasi yang diberikan, Prabowo Subianto akan menjadi presiden Indonesia berikutnya dalam Pemilihan Presiden 2024. Hasil ini diumumkan pada tanggal 14 Februari 2024. Meskipun tidak dijelaskan secara rinci alasannya, Prabowo Subianto disebut sebagai calon kuat dalam pemilihan presiden tersebut.


### GPT3: I now have final answer to the question. 